In [23]:
import re, string, collections, pickle, os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
import matplotlib as plt

In [2]:
%matplotlib inline
import utils
import importlib
importlib.reload(utils)
from utils import *

In [3]:
df = pd.read_csv('data/phase1_movie_reviews-train.csv', lineterminator='\n', names=['polarity', 'title', 'review', 'year'])

In [4]:
df.shape

(90000, 4)

In [5]:
df.head()

,polarity,title,review,year
0,negative,BRUCE LEE THE LEGEND - Baaaaaad,This was a horrible movie. Thats all I have to...,2000
1,positive,Stylish yet uneven film at an affordable price.,...As a lover of certian genres such as italia...,2001
2,positive,Masterful and Commanding,Master and Commander: The Far Side of the Worl...,2003
3,positive,Great Special Effects!! Disappointed with the ...,WHAT'S THE 411 ON THIS MOVIE?I'm an avid fan o...,2009
4,positive,Bevare! Bevare! Dracula is suspect here!,When I bought my set I went to my local retail...,2004


In [6]:
df = df[['review','polarity']]

In [7]:
df.head()

,review,polarity
0,This was a horrible movie. Thats all I have to...,negative
1,...As a lover of certian genres such as italia...,positive
2,Master and Commander: The Far Side of the Worl...,positive
3,WHAT'S THE 411 ON THIS MOVIE?I'm an avid fan o...,positive
4,When I bought my set I went to my local retail...,positive


In [8]:
for i in df.index:
    if df.at[i, 'polarity'] == 'positive':
        df.at[i, 'polarity'] = 1
    else:
        df.at[i, 'polarity'] = 0

In [9]:
df.head()

,review,polarity
0,This was a horrible movie. Thats all I have to...,0
1,...As a lover of certian genres such as italia...,1
2,Master and Commander: The Far Side of the Worl...,1
3,WHAT'S THE 411 ON THIS MOVIE?I'm an avid fan o...,1
4,When I bought my set I went to my local retail...,1


In [10]:
df.shape

(90000, 2)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df['review'].values.astype('U'), df['polarity'].values, test_size=0.2, random_state=42)

In [12]:
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [13]:
vect = CountVectorizer(tokenizer=tokenize)

In [14]:
tf_train = vect.fit_transform(X_train)
tf_test = vect.transform(X_test)

In [25]:
svd = TruncatedSVD()
reduced_tf_train = svd.fit_transform(tf_train)

In [32]:
plot_embedding(reduced_tf_train, y_train)

TypeError: 'module' object is not callable

In [28]:
p = tf_train[y_train==1].sum(0) + 1
q = tf_train[y_train==0].sum(0) + 1
r = np.log((p/p.sum())/(q/q.sum()))
b = np.log(len(p)/len(q))

In [29]:
pre_preds = tf_test @ r.T + b
preds = pre_preds.T > 0
acc = (preds==y_test).mean()
print(f'Accuracy: {acc}')

Accuracy: 0.8025555555555556


In [30]:
model = LogisticRegression(C=0.2, dual=True)
model.fit(tf_train, y_train)
preds = model.predict(tf_test)
acc = (preds==y_test).mean()
print(f'Accuracy: {acc}')

c:\users\marcus\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: Unknown label type: 'unknown'